In [1]:
#Get bankrupt 10-K reports after the downloads in extraction.ipynb
import os
root="H://DATA/bankrupt/"
os.chdir(root)
files=os.listdir()
data=[]
for file in files:
    path=root+file
    with open(path, 'r',encoding="utf-8-sig") as file:
        text = file.read().replace('\n', ' ')
        data=data+[text]

In [3]:
#Construct bankrupt part of the dataset
label=[1]*len(files)
sheet={"Text":data,"Label":label}
import pandas as pd
bankrupt=pd.DataFrame(sheet)

In [4]:
#Get healthy 10-K reports 
root="H://DATA/healthy/"
os.chdir(root)
files=os.listdir()
data=[]
for file in files:
    path=root+file
    with open(path, 'r',encoding="utf-8-sig") as file:
        text = file.read().replace('\n', ' ')
        data=data+[text]


In [5]:
#Construct healthy part of the dataset
label=[0]*len(files)
sheet={"Text":data,"Label":label}
healthy=pd.DataFrame(sheet)

In [7]:
#Concatenate the two parts to form a complete dataset
df=pd.concat([healthy, bankrupt], axis=0,ignore_index=True)
df


,Text,Label
0,10-K 1 d675550d10k.htm FORM 10-K Form 10-K T...,0
1,10-K 1 aamc-20121231x10k.htm 10-K AAMC-20...,0
2,10-K 1 a10k_123108.htm FORM 10-K Unassociate...,0
3,anik20201231_10k.htm 0000898437 Anika The...,0
4,10-K 1 d10k.htm FORM 10-K Form 10-K Table of...,0
...,...,...
2527,10-K 1 j4947_10k.htm 10-K SECURITIES ...,1
2528,10-K 1 form10k-032107.htm 032107 UNITED ST...,1
2529,10-K 1 g17282e10vk.htm FORM 10-K FORM 10-K ...,1
2530,10-K 1 a08-7760_110k.htm 10-K UNITED ...,1


In [8]:
#Lower case all text
df["Text"] = df["Text"].str.lower()

In [10]:
#Remove punctuations
import string
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

df["Text"] = df["Text"].apply(lambda text: remove_punctuation(text))
df.head()

,Text,Label
0,10k 1 d675550d10khtm form 10k form 10k table...,0
1,10k 1 aamc20121231x10khtm 10k aamc2012123...,0
2,10k 1 a10k123108htm form 10k unassociated do...,0
3,anik2020123110khtm 0000898437 anika thera...,0
4,10k 1 d10khtm form 10k form 10k table of con...,0


In [11]:
#Download NLTK stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\toby1\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
#Get NLTK stopwords
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))

"i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, and, but, if, or, because, as, until, while, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, few, more, most, other, some, such, no, nor, not, only, own, same, so, than, too, very, s, t, can, will, just, don, don't, should, should've, now, d, ll, m, o, re, ve, y, ain, aren, aren't, couldn, couldn't, didn, didn't, doesn, doesn't, hadn, hadn't, hasn, hasn't, haven, haven't, isn, isn't, ma, mightn, mightn't, mustn, mus

In [13]:
#Remove stopwords
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

df["Text"] = df["Text"].apply(lambda text: remove_stopwords(text))
df.head()

,Text,Label
0,10k 1 d675550d10khtm form 10k form 10k table c...,0
1,10k 1 aamc20121231x10khtm 10k aamc2012123110k ...,0
2,10k 1 a10k123108htm form 10k unassociated docu...,0
3,anik2020123110khtm 0000898437 anika therapeuti...,0
4,10k 1 d10khtm form 10k form 10k table contents...,0


In [ ]:
#Remove all decimals
df['Text'] = df['Text'].str.replace('\d+', '')

In [ ]:
#Remove other common unicode characters
df['Text'] = df['Text'].str.replace('\u200b', '')
df['Text'] = df['Text'].str.replace('—', '')
df['Text'] = df['Text'].str.replace('\xa0','')

In [ ]:
#Word Counter to count word occcurence
from collections import Counter
cnt = Counter()
for text in df["Text"].values:
    for word in text.split():
        cnt[word] += 1

In [ ]:
#Keep only the most common 20000 words
FREQWORDS = set([w for (w, wc) in cnt.most_common(20000)])
def keep_freqwords(text):
    """custom function to remove the frequent words"""
    return " ".join([word for word in str(text).split() if word in FREQWORDS])

df["Text"] = df["Text"].apply(lambda text: keep_freqwords(text))
df.head()

,Text,Label
0,k ddkhtm form k form k table contents united s...,0
1,k k united statessecurities exchange commissio...,0
2,k akhtm form k document united states securiti...,0
3,anika therapeutics inc false fy quarter ended ...,0
4,k dkhtm form k form k table contents index fin...,0


In [ ]:
#Download NLTK Wordnet
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\toby1\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
#Use NLTK WordnetLemmatizer to Lemmatize text
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

df["Text"] = df["Text"].apply(lambda text: lemmatize_words(text))
df.head()

,Text,Label
0,k ddkhtm form k form k table content united st...,0
1,k k united statessecurities exchange commissio...,0
2,k akhtm form k document united state security ...,0
3,anika therapeutic inc false fy quarter ended m...,0
4,k dkhtm form k form k table content index fina...,0


In [ ]:
#Defined fixed length of text segment
SEQ_LEN=1024

In [ ]:
#Truncation of text to the fixed length defined in SEQ_LEN
import functools
import operator
t = df.apply(lambda x:x.Text.split(), axis=1).apply(lambda x: [x[i * SEQ_LEN:(i + 1) * SEQ_LEN] for i in range((len(x) + SEQ_LEN - 1) // SEQ_LEN )])
df['t'] = t
l = df.apply(lambda x:[x.Label] * len(x.t), axis=1)
# flat a list and make a dataframe from it. 
df = pd.DataFrame({"Text" : functools.reduce(operator.iconcat, df.t.to_list(), []), 
              "Label" : functools.reduce(operator.iconcat, l.to_list(), [])})
df['Text'] = df['Text'].apply(' '.join)

In [ ]:
#Drop duplicates, if any
df.drop_duplicates()

,Text,Label
0,k ddkhtm form k form k table content united st...,0
1,voting common stock newsource residential simu...,0
2,hedge fund private equity fund partnership dev...,0
3,impose restriction conduct occupant home use c...,0
4,taxed reit believe residential subject federal...,0
...,...,...
18353,volumetric weighted average maturity weighted ...,1
18354,periodically ass contingent liability connecti...,1
18355,dismissal plaintiff claim failure state claim ...,1
18356,loss per share give effect dilutive potential ...,1


In [ ]:
#Output the pre-processed dataset for model input
os.chdir("H://DATA/")
df.to_csv("small.csv",encoding="utf-8-sig")